---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
S = 100         # Underlying price
K = 100         # strike price
T = 1           # years to maturity
sigma = 0.4     # volatility of underlying
r = 0.02        # risk-free rate
q = 0.04        # dividend yield
vc = 10.2       # call premium
vp = 12.2       # put premium 

# CALL IMPLIED VOLATILITY

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

def callImpliedVolatility(S, K, T, r, q, v):
    def callFunc(sigma, *args):
        S, K, T, r, q, v = args

        def f(s):
            s = s if s != 0 else 1.0e-6
            d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
            d2 = d1 - sigma * np.sqrt(T)
            return (
                np.exp(-q * T) * s * norm.cdf(d1)
                - np.exp(-r * T) * K * norm.cdf(d2)
                - v
            )

        if isinstance(S, list) or isinstance(S, np.ndarray):
            return np.array([f(s) for s in S])
        else:
            return f(S)

    inpt = (S, K, T, r, q, v)
    sol = fsolve(callFunc, 0.2, args=inpt, full_output=True)
    return sol

# Calculate implied vol
sigma_c = float(callImpliedVolatility(S, K, T, r, q, vc)[0])
print(f'Call implied volatility:\t{sigma_c:.1%}')


# Black-Scholes values at implied-volatility
indx = [
    "d1",
    "N(d1)",
    "d2",
    "N(d2)",
    "exp(-qT)S",
    "exp(-rT)K",
    "exp(-qT)SN(d1)",
    "exp(-rT)KN(d2)",
    "call value",
]
tbl = pd.DataFrame(dtype=float, index=indx, columns=["values"])
d1 = (np.log(S / K) + (r - q + 0.5 * sigma_c ** 2) * T) / (sigma_c * np.sqrt(T))
tbl.loc["d1"] = d1
tbl.loc["N(d1)"] = norm.cdf(d1)
d2 = d1 - sigma_c * np.sqrt(T)
tbl.loc["d2"] = d2
tbl.loc["N(d2)"] = norm.cdf(d2)
tbl.loc["exp(-qT)S"] = np.exp(-q * T) * S
tbl.loc["exp(-rT)K"] = np.exp(-r * T) * K
tbl.loc["exp(-qT)SN(d1)"] = np.exp(-q * T) * S * norm.cdf(d1)
tbl.loc["exp(-rT)KN(d2)"] = np.exp(-r * T) * K * norm.cdf(d2)
tbl.loc["call value"] = np.exp(-q * T) * S * norm.cdf(d1) - np.exp(
    -r * T
) * K * norm.cdf(d2)
tbl_c = tbl.round(2)
tbl_c


Call implied volatility:	28.9%


,values
d1,0.08
N(d1),0.53
d2,-0.21
N(d2),0.42
exp(-qT)S,96.08
exp(-rT)K,98.02
exp(-qT)SN(d1),50.92
exp(-rT)KN(d2),40.72
call value,10.20


# PUT IMPLIED VOLATILITY

In [3]:
def putImpliedVolatility(S, K, T, r, q, v):
    def putFunc(sigma, *args):
        S, K, T, r, q, v = args

        def f(s):
            s = s if s != 0 else 1.0e-6
            d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
            d2 = d1 - sigma * np.sqrt(T)
            return (
                np.exp(-r * T) * K * norm.cdf(-d2)
                - np.exp(-q * T) * s * norm.cdf(-d1)
                - v
            )

        if isinstance(S, list) or isinstance(S, np.ndarray):
            return np.array([f(s) for s in S])
        else:
            return f(S)

    inpt = (S, K, T, r, q, v)
    sol = fsolve(putFunc, 0.2, args=inpt, full_output=True)
    return sol


# Calculate implied vol
sigma_p = float(putImpliedVolatility(S, K, T, r, q, vp)[0])
print(f'Put implied volatility:\t{sigma_p:.1%}')


# Black-Scholes values at implied-volatility
indx = [
    "d1",
    "N(-d1)",
    "d2",
    "N(-d2)",
    "exp(-qT)S",
    "exp(-rT)K",
    "exp(-rT)KN(-d2)",
    "exp(-qT)SN(-d1)",
    "put value",
]
tbl = pd.DataFrame(dtype=float, index=indx, columns=["values"])
d1 = (np.log(S / K) + (r - q + 0.5 * sigma_p ** 2) * T) / (sigma_p * np.sqrt(T))
tbl.loc["d1"] = d1
tbl.loc["N(-d1)"] = norm.cdf(-d1)
d2 = d1 - sigma_p * np.sqrt(T)
tbl.loc["d2"] = d2
tbl.loc["N(-d2)"] = norm.cdf(-d2)
tbl.loc["exp(-qT)S"] = np.exp(-q * T) * S
tbl.loc["exp(-rT)K"] = np.exp(-r * T) * K
tbl.loc["exp(-qT)SN(-d1)"] = np.exp(-q * T) * S * norm.cdf(-d1)
tbl.loc["exp(-rT)KN(-d2)"] = np.exp(-r * T) * K * norm.cdf(-d2)
tbl.loc["put value"] = np.exp(-r * T) * K * norm.cdf(-d2) - np.exp(-q * T) * S * norm.cdf(-d1)
tbl_p = tbl.round(2)
tbl_p

Put implied volatility:	29.0%


,values
d1,0.08
N(-d1),0.47
d2,-0.21
N(-d2),0.58
exp(-qT)S,96.08
exp(-rT)K,98.02
exp(-rT)KN(-d2),57.32
exp(-qT)SN(-d1),45.12
put value,12.20


# CALCULATIONS FOR FIGURE

In [4]:
# Black-Scholes functions
def callBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-q * T) * s * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)

def putBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-r * T) * K * norm.cdf(-d2) - np.exp(-q * T) * s * norm.cdf(-d1)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


# Black-Scholes as function of volatility
maxprem = 2 * max(vc,vp)

v_c_l, sigma_c_l = [], []
sigma = 0.02
prem = 0
while prem < maxprem:
    prem = callBS(S, K, T, sigma, r, q)
    v_c_l.append(prem)
    sigma_c_l.append(sigma)
    sigma += 0.02

v_p_l, sigma_p_l = [], []
sigma = 0.02
prem = 0
while prem < maxprem:
    prem = putBS(S, K, T, sigma, r, q)
    v_p_l.append(prem)
    sigma_p_l.append(sigma)
    sigma += 0.02


# FIGURE

In [6]:
import plotly.graph_objects as go
fig = go.Figure()

trace1 = go.Scatter(
    x=sigma_c_l,
    y=v_c_l,
    mode="lines",
    name="Call",
    hovertemplate="call value = $%{y:.2f} when volatility = %{x:.0%}",
    line=dict(color='blue'),
)
trace2 = go.Scatter(
    x=sigma_p_l,
    y=v_p_l,
    mode="lines",
    name="Put",
    hovertemplate="put value = $%{y:.2f} when volatility = %{x:.0%}",
    line=dict(color='green')
)
trace3 = go.Scatter(
    x=sigma_c_l,
    y=[vc] * len(sigma_c_l),
    mode="lines",
    line=dict(dash="dot", color='blue'),
    hovertemplate="call premium = $%{y:.2f}<extra></extra>",
    showlegend=False,
)
trace4 = go.Scatter(
    x=sigma_p_l,
    y=[vp] * len(sigma_p_l),
    mode="lines",
    line=dict(dash="dot", color='green'),
    name="Put Premium",
    hovertemplate="put premium = $%{y:.2f}<extra></extra>",
    showlegend=False,
)
trace5 = go.Scatter(
    x=[sigma_c],
    y=[vc],
    mode="markers",
    hovertemplate="call implied volatility = %{x:0.1%}<extra></extra>",
    marker=dict(size=15, color='blue'),
    showlegend=False,
)
trace6 = go.Scatter(
    x=[sigma_p],
    y=[vp],
    mode="markers",
    hovertemplate="put implied volatility = %{x:0.1%}<extra></extra>",
    marker=dict(size=15, color='green'),
    showlegend=False,
)

for trace in [trace1, trace2, trace3, trace4, trace5, trace6]:
    fig.add_trace(trace)

fig.update_layout(
    yaxis = dict(
        title="Option Premium",
        tickformat=",.1f",
        range=[0, np.maximum(vc, vp) * 2]
    ),
    xaxis=dict(
        title="Volatility",
        tickformat=",.1f",
        range=[0, np.maximum(sigma_c, sigma_p) * 2]
    ),
    legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="left", 
        x=0.01),
    template="plotly_white",
)
fig.show()